In [ ]:
# !pip install webrtcvad
# !pip install pygame
# !pip install pyaudio webrtcvad 
# !pip install google-cloud-texttospeech


In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.agents.agent_toolkits import ZapierToolkit
from docx import Document
from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents import initialize_agent , create_structured_chat_agent, AgentType
import random
from groq import Groq
from QuoteUtils import CtxMgr
from groq import Groq
import random
from langchain_groq import ChatGroq
api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192")

In [ ]:
import os
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx2pdf import convert
from langchain.tools import BaseTool
import re
import json
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build

llm.groq_api_key = random.choice(api_keys)

In [ ]:
def make_docmgr_write_to_file(cc_out):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """You are a detailed and professional assistant. Your task is to generate comprehensive and detailed formatted text. 
                Ensure everything is included !!!!! DO NOT DO NOT DO NOTTTTTTT summarize or truncate any content. OR ELSE I WILL GET REALLY MAD YOU MUST INCLUDE EVERYTHING INCLUDE EVERYTHHIINGNGNGNGNGNGNGGGG!!!!!!!
                Your response must be well-formed and include all details. 

                You must ALWAYS ALWAYS ALWAYS!!!!! output in a structured format as follows AND NO QUOTATION MARKS OR BACKSLASHES!:
                
                            
                {
                    "title": "The Great Renovation",
                    "sections": [
                        {
                            "content": "<specific text content>",
                            "font_name": "<font family>",
                            "font_size": <integer specifying font size>,
                            "bold": <true or false>,
                            "italic": <true or false>,
                            "alignment": "<right, left, justify or center>"
                            "color": [<integer for red>, <integer for green>, <integer for blue>]
                        },
                        {
                            "content": "<another specific text content>",
                            "font_name": "<another font family>",
                            "font_size": <another integer specifying font size>,
                            "bold": <true or false>,
                            "italic": <true or false>,
                            "alignment": "<right, left, justify or center>"
                            "color": [<integer for red>, <integer for green>, <integer for blue>]
                        }
                        // ... more sections as needed
                    ],
                    "append": <true or false> (this indicates appending to document or not)
                }
                PLEASE! Ensure the output is well-formed and valid."""
            },
            {
                "role": "user",
                "content": "Give whatever file title you want;NO COLONS!ONLY HYPHENS!. Please Include a new title IN THE DOCUMENT, headings if you deem fit, and please format in a nice readable and coherent way.  \
                    Specify the formatting for this text. Please make it a \
                        PROFESSIONALLY formatted text WITH black & blue colors and times new roman font PLEASE. NO NO NO bullet points of ANY KIND unless specified: " + cc_out
            }
        ],
        model="llama3-70b-8192",
    )

    cc_out2 = chat_completion.choices[0].message.content

    print('CCOUT2\n',cc_out2)

    # title, sections, append = parse_formatted_response(cc_out2)

    # WriteFileTool()._run(title, sections, append)

    return cc_out2

#make_docmgr_write_to_file()


In [ ]:
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import googleapiclient.errors
from googleapiclient.errors import HttpError
import re
import time

DOCUMENT_IDS_FILE = "document_ids.txt"
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

def authenticate(credentials_path):
    """Authenticate the user with Google Drive API using OAuth 2.0."""
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credentials_path, SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# Initialize credentials and API clients
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
creds = authenticate(credentials_path)
docs_service = build('docs', 'v1', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)

def create_google_doc(title):
    body = {
        'title': title
    }
    doc = docs_service.documents().create(body=body).execute()
    with open(DOCUMENT_IDS_FILE,'w') as difile:
        difile.write(doc['documentId'])
    return doc['documentId']

def share_document_with_user(document_id, user_email):
    permission = {
        'type': 'user',
        'role': 'writer',
        'emailAddress': user_email
    }
    try:
        drive_service.permissions().create(
            fileId=document_id,
            body=permission,
            fields='id'
        ).execute()
        print(f'Document shared with {user_email}')
    except googleapiclient.errors.HttpError as error:
        print(f'An error occurred: {error}')
        if 'User message' in error.resp.reason:
            print(error.resp['User message'])

def get_document_end_index(document_id):
    doc = docs_service.documents().get(documentId=document_id).execute()
    content = doc.get('body').get('content')
    print(f'Document content: {content}')  # Debug print
    end_index = content[-1]['endIndex'] if content else 1
    print(f'Document end index: {end_index}')  # Debug print
    return end_index


def append_text_with_formatting(document_id, text, font_name, font_size, bold, italic, alignment, color, start_index):
    if not text.strip():
        return start_index  # Skip empty text

    # Ensure the start index is within valid bounds
    start_index = max(1, start_index - 1)

    # Debug print for start_index and text
    print(f'Appending text starting at index: {start_index}')
    print(f'Text to append: {text}')

    requests = [
        {
            'insertText': {
                'location': {
                    'index': start_index,
                },
                'text': text
            }
        }
    ]
    text_style = {
        'fontSize': {
            'magnitude': font_size,
            'unit': 'PT'
        },
        'weightedFontFamily': {
            'fontFamily': font_name
        },
        'bold': bold,
        'italic': italic,
        'foregroundColor': {
            'color': {
                'rgbColor': {
                    'red': color[0] / 255.0,
                    'green': color[1] / 255.0,
                    'blue': color[2] / 255.0
                }
            }
        }
    }
    requests.append({
        'updateTextStyle': {
            'range': {
                'startIndex': start_index,
                'endIndex': start_index + len(text)
            },
            'textStyle': text_style,
            'fields': 'foregroundColor,bold,italic,fontSize,weightedFontFamily'
        }
    })

    alignment_mapping = {
        "left": "START",
        "center": "CENTER",
        "right": "END",
        "justify": "JUSTIFIED"
    }

    if alignment.lower() in alignment_mapping:
        google_alignment = alignment_mapping[alignment.lower()]
        requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': start_index + len(text)
                },
                'paragraphStyle': {
                    'alignment': google_alignment
                },
                'fields': 'alignment'
            }
        })

    try:
        # Execute the batchUpdate request
        response = docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
        print(f'Batch update response: {response}')  # Debug print
    except HttpError as error:
        print(f'An error occurred during batchUpdate: {error}')
        raise  # Re-raise the error to be caught by the calling function

    return start_index + len(text)

def write_to_google_doc(document_id, sections, append):
    start_index = get_document_end_index(document_id) if append else 1
    print(f'Start index: {start_index}')  # Debug print
    for section in sections:
        start_index = append_text_with_formatting(
            document_id,
            section['content'] + "\n\n",
            section.get('font_name', 'Times New Roman'),
            section.get('font_size', 12),
            section.get('bold', False),
            section.get('italic', False),
            section.get('alignment', 'left'),
            section.get('color', [0, 0, 0]),
            start_index
        )
        print(f'New start index: {start_index}')  # Debug print
        time.sleep(1)
    print(f'Finished writing to document ID: {document_id}')  # Debug print


def parse_formatted_response(response):
    title_match = re.search(r'"title":\s*"([^"]+)"', response)
    title = title_match.group(1) if title_match else "Untitled Document"
    
    sections_matches = re.findall(r'\{([^}]+)\}', response)
    sections = []
    for match in sections_matches:
        section = {}
        content_match = re.search(r'"content":\s*"([^"]+)"', match)
        font_name_match = re.search(r'"font_name":\s*"([^"]+)"', match)
        font_size_match = re.search(r'"font_size":\s*(\d+)', match)
        bold_match = re.search(r'"bold":\s*(true|false)', match)
        italic_match = re.search(r'"italic":\s*(true|false)', match)
        alignment_match = re.search(r'"alignment":\s*"([^"]+)"', match)
        color_match = re.search(r'"color":\s*\[(\d+),\s*(\d+),\s*(\d+)\]', match)
        
        if content_match:
            content = content_match.group(1)
            # Replace double quotes indicating inches with the word "inch"
            content = re.sub(r'(?<=\d)"', ' inch', content)
            section["content"] = content
        else:
            section["content"] = ""
        
        section["font_name"] = font_name_match.group(1) if font_name_match else "Arial"
        section["font_size"] = int(font_size_match.group(1)) if font_size_match else 12
        section["bold"] = bold_match.group(1) == "true" if bold_match else False
        section["italic"] = italic_match.group(1) == "true" if italic_match else False
        section["alignment"] = alignment_match.group(1) if alignment_match else "left"
        if color_match:
            section["color"] = (int(color_match.group(1)), int(color_match.group(2)), int(color_match.group(3)))
        
        sections.append(section)
    
    append_match = re.search(r'"append":\s*(true|false)', response)
    append = append_match.group(1) == "true" if append_match else False
    
    return title, sections, append

def search_document_by_name(name):
    query = f"name = '{name}' and mimeType = 'application/vnd.google-apps.document' and trashed = false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    items = results.get('files', [])
    return items

class GoogleDocWriteTool(BaseTool):
    def __init__(self):
        super().__init__(name="GoogleDocWriteTool", description="Writes any amount of content to a Google Doc with professional formatting")

    def _run(self, input_text, user_email, append=False, document_name=None):
        try:
            # Parse the input text
            processed_input = make_docmgr_write_to_file(input_text)
            title, sections, append_response = parse_formatted_response(processed_input)
            append = append or append_response

            # Search for the document by name
            if document_name:
                documents = search_document_by_name(document_name)
                if documents:
                    document_id = documents[0]['id']
                    print(f'Found existing document with ID: {document_id}')
                else:
                    document_id = create_google_doc(document_name)
                    print(f'Document created with ID: {document_id}')
            else:
                document_id = create_google_doc(title)
                print(f'Document created with ID: {document_id}')

            # Share the document with the user's email
            print(f'Sharing document with ID: {document_id} with {user_email}')  # Debug print
            share_document_with_user(document_id, user_email)
            print(f'Document shared with {user_email}')  # Debug print

            # Write the content to the Google Doc
            print(f'Writing content to document ID: {document_id}')  # Debug print
            write_to_google_doc(document_id, sections, append)
            print(f'Content written to document ID: {document_id}')  # Debug print

            # Return the URL to access the document
            document_url = f'https://docs.google.com/document/d/{document_id}/edit'
            print(f'Access the document at: {document_url}')
            return "THE CONTENT HAS BEEN WRITTEN!!!!!!!! PLEASE STOP!!!!!!!! YOURE DONE!!!!!!!!! NO MORE!!!!"
        except HttpError as error:
            print(f'An error occurred: {error}')
            return f"An error occurred: {error}"


In [ ]:
from langchain.tools import BaseTool
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import googleapiclient
import google.auth.exceptions

class GoogleDriveUploadTool(BaseTool):
    name = "GoogleDriveUploadTool"
    description = ("Uploads a PDF to Google Drive and sets permissions for a specific user. "
                   "Please set the 'rename' parameter to None if the user does not want to rename the file before uploading "
                   "to Google Drive. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION")

    def _run(self, file_path: str, user_email: str, rename: str) -> str:
        #credentials_path = "C:\Users\THEBATMAN\Documents\GitHub\RealEstateAI\filemanager-425819-341d30387005.json"
        credentials_path = "C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-3a0e2f411c8d.json"
        # Authenticate and create the service
        try:
            credentials = service_account.Credentials.from_service_account_file(credentials_path)
            service = build('drive', 'v3', credentials=credentials)
            print("Successfully authenticated and created the service.")
        except google.auth.exceptions.GoogleAuthError as e:
            return f"Authentication error: {e}"

        # File details
        if rename is not None:
            file_metadata = {'name': rename}
        else:
            file_metadata = {'name': os.path.basename(file_path)}

        if not os.path.exists(file_path):
            return f"File not found: {file_path}"
        else:
            media = MediaFileUpload(file_path, mimetype='application/pdf')
            try:
                # Upload file
                file = service.files().create(body=file_metadata, media_body=media, fields='id, name, parents').execute()
                file_id = file.get('id')
                with open('file_id_history.txt','w') as id_hist_file:
                    id_hist_file.write(file_id)
                print(f"File ID: {file_id}")

                # Verify the upload by fetching the file details
                file_info = service.files().get(fileId=file_id, fields='id, name, parents').execute()
                print("File details:")
                print(file_info)

                # Sharing settings
                permission = {
                    'type': 'user',
                    'role': 'writer',  # Set to 'reader' if read-only access is needed
                    'emailAddress': user_email,
                    
                }
                

                # Grant permission to the specific user
                try:
                    service.permissions().create(
                        fileId=file_id,
                        body=permission,
                        fields='id',
                        sendNotificationEmail=False
                    ).execute()
                    
                    print(f"Granted {permission['role']} access to {user_email}")
                    
                    

                except googleapiclient.errors.HttpError as e:
                    print(f"Error setting permission: {e}")
                    return f"Error setting permission: {e}"

                return f"File uploaded and permissions set successfully. File ID: {file_id}. PLEASE STOP WORKING! THE TOOL HAS BEEN EXECUTED PROPERLY!" 

            except Exception as e:
                return f"Error during file upload or conversion: {e}"

    

    def _arun(self, file_path: str, user_email: str, rename: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")

# Example usage
# tool = GoogleDriveUploadTool()
# response = tool._run(file_path='./documents/The Great Renovation.pdf', user_email='specific-user@example.com', rename=None)
# print(response)

import os
import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build


class GoogleSheetsUpdateTool(BaseTool):
    name = "GoogleSheetsUpdateTool"
    description = ("Appends three columns - name, phone number, and link to a PDF - to a preexisting logging google sheet. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION")
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    #credentials_path = 'C:\Users\THEBATMAN\Documents\GitHub\RealEstateAI\filemanager-425819-341d30387005.json'
    credentials_path = "C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-3a0e2f411c8d.json"
    spreadsheet_id = '1TSFyiTwctC1tABr2RQouBzLRMCG4RZ7lXdTVi-I58Mo'
    range_name = 'Sheet1'
    
    #creds = authenticate_google_sheets()
    
    def authenticate_google_sheets(self):
        creds = service_account.Credentials.from_service_account_file(
            self.credentials_path, scopes=self.SCOPES)
        return creds
    
    def read_file_content(self, file_path):
        with open(file_path, 'r') as file:
            content = file.read().strip()
        return content
    
    
    def append_row_to_google_sheets(self, values):
        """
        Appends a row to the specified range in the Google Sheets spreadsheet.
        
        :param values: The data to append, as a list of lists (e.g., [['A1', 'B1', 'C1']]).
        """
        service = build('sheets', 'v4', credentials=self.authenticate_google_sheets())
        sheet = service.spreadsheets()
        body = {'values': values}
        result = sheet.values().append(
            spreadsheetId=self.spreadsheet_id, range=self.range_name,
            valueInputOption="RAW", body=body,
            insertDataOption="INSERT_ROWS"
        ).execute()
        print(f"{result.get('updates').get('updatedCells')} cells appended.")

    def _run(self, name: str, phone_number: str, linkstr:str):
        """
        Run the tool to append the row with the given name, phone number, and link to the PDF.
        
        :param name: The full legal name to append.
        :param phone_number: The phone number to append.
        """
        fileID = self.read_file_content('file_id_history.txt')
        
        print(f"Extracted file ID: {fileID}")
        link = f"https://drive.google.com/file/d/{fileID}/view"
        values = [[]]
        if linkstr:
            values = [[name, phone_number, linkstr]]
        else:
            values = [[name, phone_number, link]]
        self.append_row_to_google_sheets(values)
        return "Row appended successfully. YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!"

    def _arun(self, name: str, phone_number: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")
    
import os
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
from langchain.tools import BaseTool
from pydantic import BaseModel, Field, Extra

# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

class GmailSendPdfTool(BaseTool):
    name = "GmailSendPdfTool"
    description = "Sends an email with an optional PDF attachment using Gmail API."

    credentials_path: str = Field(..., description="Path to the credentials JSON file")

    class Config:
        extra = Extra.allow

    def __init__(self, credentials_path: str):
        super().__init__()
        self.credentials_path = credentials_path
        self.creds = None
        self.service = None
        self.authenticate()

    def authenticate(self):
        """Authenticate the user with Gmail API."""
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_path, SCOPES)
                self.creds = flow.run_local_server(port=0)
            
        self.service = build('gmail', 'v1', credentials=self.creds)

    def send_email(self, sender_email, recipient_email, subject, body, pdf_path=None):
        """Send an email with an optional PDF attachment."""
        message = MIMEMultipart()
        message['to'] = recipient_email
        message['from'] = sender_email
        message['subject'] = subject
        body_part = MIMEText(body, 'plain')
        message.attach(body_part)
        
        if pdf_path:
            part = MIMEBase('application', 'octet-stream')
            with open(pdf_path, 'rb') as file:
                part.set_payload(file.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(pdf_path)}"')
            message.attach(part)
       
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        raw_message = {'raw': raw_message}

        try:
            message = (self.service.users().messages().send(userId="me", body=raw_message).execute())
            print(f'Message Id: {message["id"]}')
            return message
        except HttpError as error:
            print(f'An error occurred: {error}')
            return None

    def _run(self, sender_email: str, recipient_email: str, subject: str, body: str, pdf_path: str = None):
        """Run the tool to send the email with the optional PDF attachment."""
        result = self.send_email(sender_email, recipient_email, subject, body, pdf_path)
        return "Email sent successfully. YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!" if result else "Failed to send email."

    def _arun(self):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")
    




In [ ]:
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from langchain.tools import BaseTool
from pydantic import BaseModel, Field, Extra

SCOPES = ['https://www.googleapis.com/auth/drive']

class CreateFolderTool(BaseTool):
    name = "CreateFolderTool"
    description = "Creates a new folder in Google Drive using OAuth 2.0 for secure user authentication."
    credentials_path: str = Field(..., description="Path to the credentials JSON file")

    class Config:
        extra = Extra.allow

    def __init__(self, credentials_path: str):
        super().__init__()
        self.credentials_path = credentials_path
        self.creds = None
        self.authenticate()

    def authenticate(self):
        """Authenticate the user with Google Drive API."""
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_path, SCOPES)
                self.creds = flow.run_local_server(port=0)
        
        self.service = build('drive', 'v3', credentials=self.creds)

    def search_folder(self, folder_name: str):
        """Search for a folder by name in Google Drive."""
        try:
            query = f"name = '{folder_name}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
            results = self.service.files().list(q=query, fields="files(id, name)").execute()
            items = results.get('files', [])
            if items:
                print(f'Found folder: {items[0]["name"]} (ID: {items[0]["id"]})')
                
                return items[0]['id']
            return None
        except HttpError as error:
            print(f'An error occurred during folder search: {error}')
            return None

    def create_folder(self, folder_name: str):
        """Create a folder in Google Drive."""
        folder_id = self.search_folder(folder_name)
        if folder_id:
            print(f'Folder "{folder_name}" already exists with ID: {folder_id}')
            with open('folder_ids.txt','w') as f:
                f.write(folder_id)
            return folder_id
        
        file_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }
        try:
            file = self.service.files().create(body=file_metadata, fields='id').execute()
            print(f'Folder "{folder_name}" created with ID: {file.get("id")}')
            with open('folder_ids.txt','w') as f:
                f.write(file.get('id'))
            return file.get('id')
        except HttpError as error:
            print(f'An error occurred while creating folder: {error}')
            return None

    def _run(self, folder_name: str):
        """Run the tool to create a folder in Google Drive."""
        result = self.create_folder(folder_name)
        return result if result else "Failed to create folder."

    def _arun(self):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")


class MoveFileTool(BaseTool):
    name = "MoveFileTool"
    description = "Moves a file within Google Drive using OAuth 2.0 for secure user authentication. It searches for a file by name and moves it to the specified folder."
    credentials_path: str = Field(..., description="Path to the credentials JSON file")

    class Config:
        extra = Extra.allow

    def __init__(self, credentials_path: str):
        super().__init__()
        self.credentials_path = credentials_path
        self.creds = None
        self.tmp_folder_id = None
        self.authenticate()
        #self.ensure_tmp_folder()

    def authenticate(self):
        """Authenticate the user with Google Drive API."""
        print("Authenticating with Google Drive API...")
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                print("Refreshing expired credentials...")
                self.creds.refresh(Request())
            else:
                print("Running local server for authentication...")
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_path, SCOPES)
                self.creds = flow.run_local_server(port=0)
        
        self.service = build('drive', 'v3', credentials=self.creds)
        print("Authentication successful.")

    def ensure_tmp_folder(self):
        """Ensure the temporary folder exists."""
        create_folder_tool = CreateFolderTool(credentials_path=self.credentials_path)
        self.tmp_folder_id = create_folder_tool.create_folder('tmp')

    def search_file(self, file_name: str):
        """Search for a file by name in Google Drive."""
        print(f"Searching for file: {file_name}")
        try:
            query = f"name contains '{file_name}' and trashed=false"
            print(f"Query: {query}")
            results = self.service.files().list(q=query, fields="files(id, name)").execute()
            items = results.get('files', [])
            if not items:
                print(f'No files found with the name: {file_name}')
                return None
            for item in items:
                print(f'Found file: {item["name"]} (ID: {item["id"]})')
            return items[0]['id']  # Return the first matching file ID
        except HttpError as error:
            print(f'An error occurred during file search: {error}')
            return None
  
    def move_file(self, file_id: str, folder_id: str):
        """Move the file to the new folder."""
        print(f"Moving file ID: {file_id} to folder ID: {folder_id}")
        try:
            # Retrieve the existing parents to remove
            print("Retrieving existing parents...")
            file = self.service.files().get(fileId=file_id, fields='parents').execute()
            previous_parents = file.get('parents', [])
            if not isinstance(previous_parents, list):
                print(f"Unexpected format for parents field: {previous_parents}")
                previous_parents = []
            previous_parents_str = ",".join(previous_parents)
            print(f"Previous parents: {previous_parents_str}")

            # if not previous_parents_str:
            #     # Add the file to a temporary parent folder first
            #     print(f"No previous parents. Adding file to temporary folder first...")
            #     self.service.files().update(
            #         fileId=file_id,
            #         addParents=self.tmp_folder_id,
            #         fields='id, parents'
            #     ).execute()
            #     previous_parents_str = self.tmp_folder_id

            # Move the file to the new folder
            print("Updating file to new folder...")
            file = self.service.files().update(
                fileId=file_id,
                addParents=folder_id,
                removeParents=previous_parents_str,
                fields='id, parents'
            ).execute()
            print(f'Moved file {file_id} to folder {folder_id}')
            return f'Moved file {file_id} to folder {folder_id}. PLEEEASEEEE STOOOPPPPPP!!!!!! NO MORE!!!!!!! YOU ARE DONE!!!!'
        except HttpError as error:
            print(f'An error occurred during file move: {error}')
            return None
        except Exception as e:
            print(f'An unexpected error occurred: {e}')
            return None

    def search_folder(self, folder_name: str):
        """Search for a folder by name in Google Drive using a contains query."""
        try:
            query = f"name contains '{folder_name}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
            results = self.service.files().list(q=query, fields="files(id, name)").execute()
            items = results.get('files', [])
            if items:
                for item in items:
                    print(f'Found folder: {item["name"]} (ID: {item["id"]})')
                return items[0]['id']
            return None
        except HttpError as error:
            print(f'An error occurred during folder search: {error}')
            return None

    def _run(self, file_name: str, folder_name: str):
        """Run the tool to search for the file and move it to the specified folder."""
        folder_id = self.search_folder(folder_name)
        if folder_id is None:
            return "Folder not found."
        
        print(f"Running MoveFileTool with file_name: {file_name} and folder_id: {folder_id}")
        file_id = self.search_file(file_name)
        if file_id is None:
            return "File not found."
        
        result = self.move_file(file_id, folder_id)
        return result if result else "Failed to move file."

    def _arun(self):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")


In [ ]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from langchain.tools import BaseTool
from pydantic import Field, Extra
import os

SCOPES = ['https://www.googleapis.com/auth/drive']

class FolderMovementTool(BaseTool):
    name = "Folder movement tool"
    description = "Manages folders in Google Drive using OAuth 2.0 for secure user authentication. Provides functionality to move folders and their contents."
    credentials_path: str = Field(..., description="Path to the credentials JSON file")

    class Config:
        extra = Extra.allow

    def __init__(self, credentials_path: str):
        super().__init__()
        self.credentials_path = credentials_path
        self.creds = None
        self.authenticate()

    def authenticate(self):
        """Authenticate the user with Google Drive API using OAuth 2.0."""
        if os.path.exists('token.json'):
            self.creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_path, SCOPES)
                self.creds = flow.run_local_server(port=0)
            with open('token.json', 'w') as token:
                token.write(self.creds.to_json())
        self.service = build('drive', 'v3', credentials=self.creds)

    def search_folder(self, folder_name: str):
        """Search for a folder by name in Google Drive using a contains query."""
        try:
            query = f"name contains '{folder_name}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
            results = self.service.files().list(q=query, fields="files(id, name)").execute()
            items = results.get('files', [])
            if items:
                for item in items:
                    print(f'Found folder: {item["name"]} (ID: {item["id"]})')
                return items[0]['id']
            return None
        except HttpError as error:
            print(f'An error occurred during folder search: {error}')
            return None
        
    def move_file_to_folder(self, file_id, folder_id):
        try:
            # Retrieve the existing parents to remove
            file = self.service.files().get(fileId=file_id, fields='parents').execute()
            previous_parents = ",".join(file.get('parents'))

            # Move the file to the new folder
            self.service.files().update(
                fileId=file_id,
                addParents=folder_id,
                removeParents=previous_parents,
                fields='id, parents'
            ).execute()
            print(f'Moved file {file_id} to folder {folder_id}')
        except HttpError as error:
            print(f'An error occurred: {error}')

    def move_folder_and_contents(self, folder_id, new_parent_folder_id):
        service = build('drive', 'v3', credentials=self.creds)
        try:
            # Move the folder itself
            self.move_file_to_folder(folder_id, new_parent_folder_id)

            # List all files and subfolders in the folder
            query = f"'{folder_id}' in parents"
            results = service.files().list(q=query, fields="files(id, mimeType)").execute()
            items = results.get('files', [])

            for item in items:
                if item['mimeType'] == 'application/vnd.google-apps.folder':
                    # Recursively move subfolders
                    self.move_folder_and_contents(item['id'], folder_id)
                else:
                    # Move files
                    self.move_file_to_folder(item['id'], folder_id)

            print(f'Moved folder {folder_id} and all its contents to {new_parent_folder_id}')
        except HttpError as error:
            print(f'An error occurred during file move: {error}')

    def move_folder_and_contents(self, folder_id, new_parent_folder_id):
        try:
            # Move the folder itself
            self.move_file_to_folder(folder_id, new_parent_folder_id)

            # List all files and subfolders in the folder
            query = f"'{folder_id}' in parents"
            results = self.service.files().list(q=query, fields="files(id, mimeType)").execute()
            items = results.get('files', [])

            for item in items:
                if item['mimeType'] == 'application/vnd.google-apps.folder':
                    # Recursively move subfolders
                    self.move_folder_and_contents(item['id'], folder_id)
                else:
                    # Move files
                    self.move_file_to_folder(item['id'], folder_id)

            print(f'Moved folder {folder_id} and all its contents to {new_parent_folder_id}')
        except HttpError as error:
            print(f'An error occurred: {error}')

    def _run(self, folder_name: str, new_parent_folder_name: str):
        """Run the tool to search for a folder by name and move its contents to a new parent folder."""

        new_parent_folder_id = self.search_folder(new_parent_folder_name)
        if new_parent_folder_id is None:
            return "NEW PARENT FOLDER not found."
        
        print(f"Running FolderManagementTool with folder_name: {folder_name} and new_parent_folder_id: {new_parent_folder_id}")
        folder_id = self.search_folder(folder_name)
        if folder_id is None:
            return "Folder not found."
        
        self.move_folder_and_contents(folder_id, new_parent_folder_id)
        return "Folder and contents moved successfully. PLEASE STOPPPPPPPPP!!!!!!!!!! RIGHT NOW!!!!!! YOU ARE DONE!!!!!!!! STOP!!!!!!!!"

    def _arun(self):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")


In [18]:
from flask import Flask, request, jsonify, send_file, render_template
import whisper
import pyaudio
import wave
import webrtcvad
import collections
import base64
from google.cloud import texttospeech
import json
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
import aiofiles
from aiogoogle import Aiogoogle
from flask_cors import CORS
import requests
import json
from langchain.agents import Tool
my_tools = []



my_tools.append(
    
    GoogleDocWriteTool()
)

my_tools.append(
    
    GoogleDriveUploadTool()
)

my_tools.append(
    
    GoogleSheetsUpdateTool()
)
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
my_tools.append(
    GmailSendPdfTool(credentials_path)
)

my_tools.append(
    MoveFileTool(credentials_path)
)

my_tools.append(
    CreateFolderTool(credentials_path)
)

my_tools.append(
    FolderMovementTool(credentials_path)
)

llm.groq_api_key = random.choice(api_keys)
agent = initialize_agent(my_tools,llm,handle_parsing_errors=True,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


from flask import Flask, request, jsonify, send_file, render_template
import whisper
import pyaudio
import wave
import webrtcvad
import collections
import base64
from google.cloud import texttospeech
import json
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
from flask_cors import CORS
import uuid
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(threadName)s] %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Global variables
chat_history = []
model = whisper.load_model("base")
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 1000000
vad = webrtcvad.Vad(3)
audio = pyaudio.PyAudio()
credentials = {"name": "", "email": "", "recemail": "", "phone": ""}
from flask_socketio import SocketIO, emit

#tts_service_acct_path = 'C:\\Users\\THEBATMAN\\Documents\\GitHub\\RealEstateAI\\filemanager-425819-341d30387005.json'
tts_service_acct_path = "C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-3a0e2f411c8d.json"
tts_client = texttospeech.TextToSpeechClient.from_service_account_file(tts_service_acct_path)

executor = ThreadPoolExecutor(max_workers=5)

socketio = SocketIO(app, cors_allowed_origins="*")

is_recording = False

@app.route('/start_recording', methods=['POST'])
def start_recording():
    global is_recording
    is_recording = True
    record_audio()
    return jsonify({"status": "recording started"})

@app.route('/stop_recording', methods=['POST'])
def stop_recording():
    global is_recording
    is_recording = False
    return jsonify({"status": "recording stopped"})

def record_audio():
    global is_recording
    logger.debug('Starting audio recording...')
    try:
        stream = audio.open(format=FORMAT, channels=CHANNELS,
                            rate=RATE, input=True,
                            frames_per_buffer=CHUNK)
        frames = []
        ring_buffer = collections.deque(maxlen=100)
        triggered = False
        voiced_frames = []
        silence_threshold = 10
        silence_chunks = 0

        while is_recording:
            data = stream.read(CHUNK)
            frames.append(data)

            num_subframes = int(len(data) / 320)
            for i in range(num_subframes):
                subframe = data[i*320:(i+1)*320]
                is_speech = vad.is_speech(subframe, RATE)
                ring_buffer.append((subframe, is_speech))

            num_voiced = len([f for f, speech in ring_buffer if speech])

            if not triggered:
                if num_voiced > 0.6 * ring_buffer.maxlen:
                    triggered = True
                    voiced_frames.extend([f for f, s in ring_buffer])
                    ring_buffer.clear()
            else:
                voiced_frames.append(data)
                if num_voiced < 0.2 * ring_buffer.maxlen:
                    silence_chunks += 1
                    if silence_chunks > silence_threshold:
                        triggered = False
                        break
                else:
                    silence_chunks = 0

        stream.stop_stream()
        stream.close()

        with wave.open("C:\\DEV\\webdevfolder\\output.wav", 'wb') as wf:
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(audio.get_sample_size(FORMAT))
            wf.setframerate(RATE)
            wf.writeframes(b''.join(voiced_frames))
        logger.debug('Audio recording completed and file saved.')
    except Exception as e:
        logger.error(f"An error occurred while recording audio: {e}")

def transcribe_audio():
    result = model.transcribe("C:\\DEV\\webdevfolder\\output.wav")
    transcription = result['text']
    
    logger.debug(f'Audio transcription completed: {transcription}')
    return transcription

async def ai_response(transcription: str):
    global chat_history
    logger.debug(f'Generating AI response for transcription: {transcription}')
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """You are a nice, great assistant. User will tell you things. You just respond. 
                YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE EVERYTHING YOU NEED.
                DO NOT ASSUME USER WANTS TO WRITE TO A DOCUMENT OR DO ANYTHING ELSE UNLESS YOU ARE 100% SURE!!!!!UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY
                If what the user says is one of these  you must explicitly say AT THE END OF YOUR RESPONSE in this very format depending on which tool - "I will use the [google doc write tool, gmail tool, sheets tool, move file to a folder tool, create folder tool, the move folder tool, OR drive upload tool]". 
                so that user can confirm if you got it correctly. 
                When writing to a file, just start ONLY ONLY ONLY THE BEGINNING OF THE ENTIRE CONTENT with '@' symbol when generating file content. There should only be ONNNEEEEE '@' symbol in response and that is at the beginning!!!!!
                WHEN WRITING TO FILE--> If user indicates that they want to add on to existing content, \
                you must say 'yes append' before the aforementioned '@' symbol. OTHERWISE YOU SAY 'no append' before the aforementioned '@' symbol
                
                IMPORTANT: IF YOU REMEMBER YOU HAVE USED A TOOL ALREADY (REFER TO CHAT HISTORY), AND YOU ARE IN THE PROCESS OF USING IT AGAIN, \
                YOU MUST ASK USER IF THEY ARE SURE TO USE THE TOOL AGAIN OR NOT.

                If you have to use a google related tool, you MUST prompt the user or remind user to enter credentials in the doc manager extension. 
                If user tells you to do something that is not one of these, you kindly say that you don't have access to that functionality.
                """
            },
            {
                "role": "user",
                "content": transcription + "Here is the chat history for context (NEVER TALK ABOUT CHAT HISTORY IT IS ONLY FOR YOU NEVER TALK ABOUT IT IN YOUR RESPONSES!!!!): [" + str(chat_history) + "]"
            }
        ],
        model="llama3-70b-8192",
    )
    response = chat_completion.choices[0].message.content
    logger.debug(f'AI response generated: {response}')
    chat_history.append("USER: " + transcription + "\nTHE AI MODEL: " + response + "\n")
    
    
    asyncio.create_task(handle_response_with_agents(response))
   
    await synthesize_speech(response)

    return response

async def handle_response_with_agents(response):
    logger.debug(f'Handling response with agents: {response}')
    await asyncio.get_event_loop().run_in_executor(executor, handle_agents, response)

def handle_agents(response):
    logger.debug(f'Processing response with agents: {response}')
    name = credentials['name']
    email = credentials['email']
    recemail = credentials["recemail"]
    phone = credentials['phone']
    print(email, recemail, name, phone)

    title_of_file = ''
    with open('most_recent_file_created.txt') as mostrec:
        title_of_file = mostrec.read()
    thepath = f"./documents/{title_of_file}.pdf"

    FILE_ID = ''
    with open('document_ids.txt','r') as f:
        FILE_ID = f.read()

    FOLDER_ID = ''
    with open('folder_ids.txt','r') as f:
        FOLDER_ID = f.read()

    if 'I will use' in response:
        if 'google doc' in response.lower() and 'write' in response.lower():
            idx = response.find('@')
            modified_response = response[idx + 1:].strip()
            print('MODIFIED RESPONSE = ', modified_response)
            
            if('no append') in response.lower():
                append = False
            else:
                append = True
            llm.groq_api_key = random.choice(api_keys)
            print('USER EMAIL HELLLOOOOOO!!!!!!!!!!',credentials['email'])
            agent.invoke({"input": f"use ONLY google doc write tool!!! INPUT EVERYTHING CORRECTLY EVEN IF ITS LONG!!!!!!!! INPUT ONLY WHAT YOURE GIVEN ! NO CREATIVITIES!!!! append-->true or false lowercase only!!! NO DICTIONARIES, JUST EVERYTHING PASSED AS IT ISSS!!!!!--> INPUT_TEXT: {modified_response} | APPEND: {append}  USER_EMAIL: {credentials['email']},"})
        elif 'sheet' in response:
            agent.invoke({"input": f"please ONLY ONLY use google sheets update tool, input ONLY ONLY these into the tool PROPERLY! --> {credentials['name']}, {credentials['phone']}"})
        elif 'upload' in response:
            agent.invoke({"input": f"please use google drive upload tool, INPUTS --> FILE_PATH: {thepath} USER_EMAIL: {credentials['email']}"})
        elif 'mail' in response:
            if 'pdf' in response or 'PDF' in response or 'file' in response:
                agent.invoke({"input": f"please use ONLY ONLY gmail send pdf tool. INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {credentials['email']}, RECIPIENT EMAIL: {credentials['recemail']}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: {thepath}"})
            else:
                agent.invoke({"input": f"please use ONLY ONLY gmail send pdf tool. INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {credentials['email']}, RECIPIENT EMAIL: {credentials['recemail']}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: nothing"})
        elif 'create' in response and 'folder' in response:
            agent.invoke({"input": f"please ONLY use create folder tool!!! INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER NAME: the folder name as specified in response --> {response}"})
        elif 'move' in response and 'file' in response:
            agent.invoke({"input": f"please ONLY use move file tool!!! INPUT DIRECTLY THE FOLLOWING INPUTS --> FILE_NAME please deduce from the response and then pass it into tool: {response} ; FOLDER_NAME deduce from the response and then pass it into tool: {response}"})
        elif 'folder' in response and 'move' in response:
            agent.invoke({"input": f"please ONLY use move folder tool!!! INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER_NAME please deduce from the response and then pass it into tool: {response} ; NEW_PARENT_FOLDER_NAME also deduce from response: {response}"})


async def synthesize_speech(text):
    logger.debug(f'Starting speech synthesis for text: {text}')
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Casual-K"
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = await asyncio.get_event_loop().run_in_executor(
        executor, lambda: tts_client.synthesize_speech(
            input=synthesis_input, voice=voice, audio_config=audio_config
        )
    )
    async with aiofiles.open("C:\\DEV\\webdevfolder\\synthesis.mp3", 'wb') as out:
        await out.write(response.audio_content)
    print('Audio content written to file "synthesis.mp3"')
    logger.debug('Speech synthesis completed and file saved.')


@app.route('/set_credentials', methods=['POST'])
def set_credentials():
    global credentials
    data = request.get_json()
    if not data:
        return jsonify({"status": "failed", "message": "No data received"}), 400
    credentials['name'] = data.get('name')
    credentials['email'] = data.get('email')
    credentials['recemail'] = data.get('recemail')
    credentials['phone'] = data.get('phone')
    logger.info("THE CREDENTIALS ****** -------------> ", credentials)
    return jsonify({"status": "success"})


@app.route('/')
def index():
    return render_template('index2.html')

@app.route('/voice_assistant')
def voice_assistant():
    return render_template('index2.html')


@app.route('/authenticate', methods=['POST'])
def authenticate():
    auth_header = request.headers.get('Authorization')
    token = auth_header.split(' ')[1] if auth_header else None

    if not token:
        return jsonify({'error': 'Missing token'}), 400

    response = requests.get(
        'https://www.googleapis.com/oauth2/v3/userinfo',
        headers={'Authorization': f'Bearer {token}'}
    )

    if response.status_code != 200:
        return jsonify({'error': 'Failed to fetch user info'}), response.status_code

    user_info = response.json()
    return jsonify(user_info), 200

@app.route('/talk', methods=['POST'])
async def talk():
    loop = asyncio.get_event_loop()
    
    # Ensure recording is stopped before proceeding
    global is_recording
    if is_recording:
        return jsonify({"error": "Recording is still in progress"}), 400
    
    logger.debug('Starting audio transcription...')
    transcription = await loop.run_in_executor(executor, transcribe_audio)
    logger.debug(f'Audio transcription completed: {transcription}')
    
    logger.debug('Generating AI response...')
    ai_resp = await ai_response(transcription)
    logger.debug(f'AI response generated: {ai_resp}')
    
    return jsonify({'response': ai_resp})


@app.route('/get_audio')
def get_audio():
    return send_file("C:\\DEV\\webdevfolder\\synthesis.mp3", mimetype="audio/mp3")

if __name__ == '__main__':
    app.run()

2024-06-13 22:38:48,374 [ThreadPoolExecutor-12_0] DEBUG: Audio transcription completed:  Please move the folder called A2 into the folder called to the folder called Wait a moment wait a moment I'm thinking of the name to the folder called matrix class S-O-L
2024-06-13 22:38:48,374 [ThreadPoolExecutor-14_0] DEBUG: Audio transcription completed:  Please move the folder called A2 into the folder called to the folder called Wait a moment wait a moment I'm thinking of the name to the folder called matrix class S-O-L
2024-06-13 22:38:48,374 [ThreadPoolExecutor-14_0] DEBUG: Generating AI response...
2024-06-13 22:38:48,374 [ThreadPoolExecutor-14_0] DEBUG: Generating AI response for transcription:  Please move the folder called A2 into the folder called to the folder called Wait a moment wait a moment I'm thinking of the name to the folder called matrix class S-O-L
2024-06-13 22:38:48,374 [ThreadPoolExecutor-14_0] DEBUG: Request options: {'method': 'post', 'url': '/openai/v1/chat/completions'

sriramnallani35@gmail.com thegreatchrisjohn@gmail.com Sriram Nallani 5713209890


> Entering new AgentExecutor chain...


2024-06-13 22:38:49,302 [ThreadPoolExecutor-14_0] DEBUG: Speech synthesis completed and file saved.
2024-06-13 22:38:49,302 [ThreadPoolExecutor-14_0] DEBUG: AI response generated: I understand that you want to move the folder called A2 into the folder called SOL. I will use the move folder tool.
2024-06-13 22:38:49,302 [Thread-60 (process_request_thread)] INFO: 127.0.0.1 - - [13/Jun/2024 22:38:49] "POST /talk HTTP/1.1" 200 -


Audio content written to file "synthesis.mp3"


2024-06-13 22:38:49,699 [ThreadPoolExecutor-12_0] DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 14 Jun 2024 02:38:43 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Cache-Control', b'private, max-age=0, no-store, no-cache, must-revalidate'), (b'vary', b'Origin'), (b'x-ratelimit-limit-requests', b'14400'), (b'x-ratelimit-limit-tokens', b'6000'), (b'x-ratelimit-remaining-requests', b'14398'), (b'x-ratelimit-remaining-tokens', b'4547'), (b'x-ratelimit-reset-requests', b'11.614999999s'), (b'x-ratelimit-reset-tokens', b'14.521999999s'), (b'x-request-id', b'req_01j0a9s3zwf4fbcxafe874kf97'), (b'via', b'1.1 google'), (b'alt-svc', b'h3=":443"; ma=86400'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8936f21fbdc6066c-IAD'), (b'Content-Encoding', b'gzip')])
2024-06-13 22:38:49,699 [ThreadPoolExecutor-12_0] INFO: HTTP Request: POST https://api.gro

...

2024-06-13 22:38:52,143 [ThreadPoolExecutor-12_0] DEBUG: URL being requested: GET https://www.googleapis.com/drive/v3/files/1Ibjkq4tsE-YcQCLmwiPPmEaA3TFKZ75F?fields=parents&alt=json
2024-06-13 22:38:52,282 [ThreadPoolExecutor-12_0] DEBUG: URL being requested: PATCH https://www.googleapis.com/drive/v3/files/1Ibjkq4tsE-YcQCLmwiPPmEaA3TFKZ75F?addParents=1JTb3fYfpKGjFfbotxhSPWqPlaiwUJafr&removeParents=1JTb3fYfpKGjFfbotxhSPWqPlaiwUJafr&fields=id%2C+parents&alt=json
2024-06-13 22:38:52,662 [ThreadPoolExecutor-12_0] DEBUG: Request options: {'method': 'post', 'url': '/openai/v1/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nGoogleDocWriteTool: Writes any amount of content to a Google Doc with professional formatting, args: {\'input_text\': {\'title\': \'Input Text\'}, \'user_email\': {\'title\': \'User Email\'}, \'append\': {\'title\': \'Append\'

Moved file 1Ibjkq4tsE-YcQCLmwiPPmEaA3TFKZ75F to folder 1JTb3fYfpKGjFfbotxhSPWqPlaiwUJafr
Moved folder 1JTb3fYfpKGjFfbotxhSPWqPlaiwUJafr and all its contents to 1cPj_c2lfVthQfnBCC-Bx-K156YYp1qwX

Observation: Folder and contents moved successfully. PLEASE STOPPPPPPPPP!!!!!!!!!! RIGHT NOW!!!!!! YOU ARE DONE!!!!!!!! STOP!!!!!!!!
Thought:

2024-06-13 22:38:53,178 [ThreadPoolExecutor-12_0] DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 14 Jun 2024 02:38:47 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Cache-Control', b'private, max-age=0, no-store, no-cache, must-revalidate'), (b'vary', b'Origin'), (b'x-ratelimit-limit-requests', b'14400'), (b'x-ratelimit-limit-tokens', b'6000'), (b'x-ratelimit-remaining-requests', b'14397'), (b'x-ratelimit-remaining-tokens', b'3843'), (b'x-ratelimit-reset-requests', b'14.37s'), (b'x-ratelimit-reset-tokens', b'21.563999999s'), (b'x-request-id', b'req_01j0a9s7hbefcs8s561s448xxh'), (b'via', b'1.1 google'), (b'alt-svc', b'h3=":443"; ma=86400'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8936f2366e78066c-IAD'), (b'Content-Encoding', b'gzip')])
2024-06-13 22:38:53,178 [ThreadPoolExecutor-12_0] INFO: HTTP Request: POST https://api.groq.com/o

I understand that you want me to stop. I will not perform any more actions.

Action:
```
{
  "action": "Final Answer",
  "action_input": "Folder moved successfully."
}
```


> Finished chain.


2024-06-13 22:39:05,469 [Thread-64 (process_request_thread)] INFO: 127.0.0.1 - - [13/Jun/2024 22:39:05] "GET /get_audio?1718332729301 HTTP/1.1" 206 -
